## Imports

In [1]:
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.model import make_model, freeze_all_vgg, unfreeze_last_vgg

In [2]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
SEED = None

#Model parameters:
N_HIDDEN = 512

#Train parameters:
BASE_LEARNING_RATE = 0.001
FINE_TUNING_LEARNING_RATE = 0.001
INITIAL_EPOCHS = 30
FINE_TUNING_EPOCHS = 30
FINE_TUNE_AT_LAYER = 15
LOG_DIR = Path(r'../models/vgg16/logs')
SAVE_DIR = Path(r'../models/vgg16/checkpoints/trained_weights')

## DATASET

In [3]:
train_path = Path(r'../data/train')
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_path, image_size=(IMG_HEIGHT, IMG_WIDTH),\
                                                               batch_size=BATCH_SIZE, shuffle=True, \
                                                               label_mode='categorical', seed=SEED)

valid_path = Path(r'../data/valid')
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(valid_path, image_size=(IMG_HEIGHT, IMG_WIDTH),\
                                                               batch_size=BATCH_SIZE, shuffle=True, \
                                                               label_mode='categorical', seed=SEED)

class_names = train_ds.class_names
assert class_names == valid_ds.class_names
def optimize_dataset(ds):
    AUTOTUNE = tf.data.AUTOTUNE
    ds = ds.cache()
    #ds = ds.shuffle(num_examples)
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = optimize_dataset(train_ds)
valid_ds = optimize_dataset(valid_ds)

Found 1170 files belonging to 3 classes.
Found 251 files belonging to 3 classes.


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[np.argmax(labels.numpy()[i], axis=0)])
        plt.axis("off")

## Model

In [4]:
model = make_model(n_classes=len(class_names), n_hidden=N_HIDDEN)
freeze_all_vgg(model)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=BASE_LEARNING_RATE),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

#### Classifier initial training

In [5]:
tb = TensorBoard(log_dir=LOG_DIR)
checkpoint = ModelCheckpoint(r'../models/vgg16/checkpoints/train_{epoch}.tf', verbose=1, save_weights_only=True,\
                             save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1)

history = model.fit(train_ds, epochs=INITIAL_EPOCHS, validation_data=valid_ds, callbacks=[tb, checkpoint, reduce_lr, early_stopping])

Epoch 1/30
19/19 [==============================] - ETA: 0s - loss: 1.3559 - accuracy: 0.4812
Epoch 00001: val_loss improved from inf to 3.34829, saving model to ../models/vgg16/checkpoints\train_1.tf
19/19 [==============================] - 46s 2s/step - loss: 1.3559 - accuracy: 0.4812 - val_loss: 3.3483 - val_accuracy: 0.3625 - lr: 0.0010
Epoch 2/30
19/19 [==============================] - ETA: 0s - loss: 1.0801 - accuracy: 0.5547
Epoch 00002: val_loss improved from 3.34829 to 2.31936, saving model to ../models/vgg16/checkpoints\train_2.tf
19/19 [==============================] - 10s 524ms/step - loss: 1.0801 - accuracy: 0.5547 - val_loss: 2.3194 - val_accuracy: 0.5100 - lr: 0.0010
Epoch 3/30
19/19 [==============================] - ETA: 0s - loss: 0.9568 - accuracy: 0.5949
Epoch 00003: val_loss improved from 2.31936 to 1.56789, saving model to ../models/vgg16/checkpoints\train_3.tf
19/19 [==============================] - 10s 526ms/step - loss: 0.9568 - accuracy: 0.5949 - val_loss: 

Epoch 26/30
19/19 [==============================] - ETA: 0s - loss: 0.5398 - accuracy: 0.7402
Epoch 00026: val_loss did not improve from 1.00770
19/19 [==============================] - 10s 517ms/step - loss: 0.5398 - accuracy: 0.7402 - val_loss: 1.1774 - val_accuracy: 0.4582 - lr: 1.0000e-06
Epoch 27/30
19/19 [==============================] - ETA: 0s - loss: 0.5527 - accuracy: 0.7060
Epoch 00027: val_loss did not improve from 1.00770
19/19 [==============================] - 10s 518ms/step - loss: 0.5527 - accuracy: 0.7060 - val_loss: 1.1851 - val_accuracy: 0.4582 - lr: 1.0000e-06
Epoch 00027: early stopping


#### Fine tuning

In [6]:
unfreeze_last_vgg(model, which_freeze=FINE_TUNE_AT_LAYER)

total_epochs = INITIAL_EPOCHS + FINE_TUNING_EPOCHS
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNING_LEARNING_RATE),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = model.fit(train_ds, epochs=total_epochs, validation_data=valid_ds, callbacks=[tb, checkpoint, reduce_lr, early_stopping], \
                    initial_epoch=history.epoch[-1])

model.save_weights(SAVE_DIR)

Epoch 27/60
19/19 [==============================] - ETA: 0s - loss: 1.1763 - accuracy: 0.4940
Epoch 00027: val_loss did not improve from 1.00770
19/19 [==============================] - 15s 600ms/step - loss: 1.1763 - accuracy: 0.4940 - val_loss: 6.8325 - val_accuracy: 0.4303 - lr: 0.0010
Epoch 28/60
19/19 [==============================] - ETA: 0s - loss: 1.0992 - accuracy: 0.4641
Epoch 00028: val_loss did not improve from 1.00770
19/19 [==============================] - 11s 577ms/step - loss: 1.0992 - accuracy: 0.4641 - val_loss: 3.9339 - val_accuracy: 0.3386 - lr: 0.0010
Epoch 29/60
19/19 [==============================] - ETA: 0s - loss: 0.9709 - accuracy: 0.5145
Epoch 00029: val_loss did not improve from 1.00770
19/19 [==============================] - 11s 576ms/step - loss: 0.9709 - accuracy: 0.5145 - val_loss: 1.4899 - val_accuracy: 0.4741 - lr: 0.0010
Epoch 30/60
19/19 [==============================] - ETA: 0s - loss: 0.8901 - accuracy: 0.5453
Epoch 00030: val_loss did not im

Epoch 53/60
19/19 [==============================] - ETA: 0s - loss: 0.6869 - accuracy: 0.6248
Epoch 00053: val_loss did not improve from 0.85914
19/19 [==============================] - 11s 576ms/step - loss: 0.6869 - accuracy: 0.6248 - val_loss: 0.8592 - val_accuracy: 0.5299 - lr: 1.0000e-06
Epoch 54/60
19/19 [==============================] - ETA: 0s - loss: 0.7195 - accuracy: 0.6188
Epoch 00054: val_loss did not improve from 0.85914
19/19 [==============================] - 11s 576ms/step - loss: 0.7195 - accuracy: 0.6188 - val_loss: 0.8596 - val_accuracy: 0.5299 - lr: 1.0000e-06
Epoch 55/60
19/19 [==============================] - ETA: 0s - loss: 0.7047 - accuracy: 0.6274
Epoch 00055: val_loss did not improve from 0.85914
19/19 [==============================] - 11s 577ms/step - loss: 0.7047 - accuracy: 0.6274 - val_loss: 0.8601 - val_accuracy: 0.5378 - lr: 1.0000e-06
Epoch 56/60
19/19 [==============================] - ETA: 0s - loss: 0.7091 - accuracy: 0.6085
Epoch 00056: val_los

#### Model evaluation

In [10]:
test_path = Path(r'../data/test')
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_path, image_size=(IMG_HEIGHT, IMG_WIDTH), \
                                                              batch_size=BATCH_SIZE, shuffle=False, \
                                                              label_mode='categorical')

metrics = model.evaluate(test_ds)
print('Loss: {} --------- Accuracy: {}%'.format(metrics[0], np.round(metrics[1]*100, 2)))

y_pred = model.predict(test_ds)
y_pred = np.argmax(y_pred, axis=1)
y_true = tf.concat([y for x, y in test_ds], axis=0)
y_true = np.argmax(y_true.numpy(), axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=2))

pred_labels = [('PRED_' + class_name) for class_name in class_names]
real_labels = [('REAL_' + class_name) for class_name in class_names]
pd.DataFrame(confusion_matrix(y_true, y_pred), columns=pred_labels, index=real_labels)

Found 252 files belonging to 3 classes.
4/4 [==============================] - 2s 450ms/step - loss: 0.8608 - accuracy: 0.5397
Loss: 0.8608231544494629 --------- Accuracy: 53.97%
              precision    recall  f1-score   support

           0       0.74      0.84      0.79        96
           1       0.41      0.34      0.38        87
           2       0.36      0.36      0.36        69

    accuracy                           0.54       252
   macro avg       0.50      0.52      0.51       252
weighted avg       0.52      0.54      0.53       252



,PRED_0,PRED_1,PRED_2
REAL_0,81,11,4
REAL_1,16,30,41
REAL_2,12,32,25
